In [1]:
 %reload_ext autoreload

In [4]:
%autoreload

from model.base import Base
from model.head import Head
import torch
import h5py
import numpy as np
import os
from utils import build_h5,extract_features, get_model_from_ckpt,train,eval,lr_scheduler, makedirs,save_model,logging
from torch.optim import Adam
import gc
import re
from dataloader import get_dataloader

hard_dp = False
if hard_dp:
    dropout = [0.5,0.5,0.75]
else:
    dropout = [0.25,0.25,0.5]

base_model_type = 'inceptionv3'
num_level = 10
feature_type = 'wide'
head_type = 'multi_3FC'
if feature_type == 'wide':
    head_type = 'pool_3FC'
    
resize = True
augment = True
root = './experiment/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
folder = makedirs(root,base_model_type,num_level,feature_type,head_type,resize,augment,hard_dp)

splits = ['train', 'val', 'test']
h5_paths = [os.path.join(folder,split + '_fea.h5') for split in splits]

EXTRACT FEATURES

In [5]:
extract_batch_size = 100

imgloaders = get_dataloader(data_type='img',resize=resize,augment=augment,batch_size = extract_batch_size, head_type = head_type)

bmodel = Base(model = base_model_type, level = num_level)
bmodel.to(device)

for h5_path,imgloader in zip(h5_paths,imgloaders):
    h5_file = build_h5(folder,h5_path,imgloader,bmodel.channel_size,feature_type,head_type,num_level)
    extract_features(imgloader, device, bmodel, feature_type, h5_file,extract_batch_size,head_type,num_level)

del bmodel
gc.collect()

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to ./pretrained_model\hub\checkpoints\inception_v3_google-1a9a5a14.pth


3877

TRAIN HEAD

In [7]:
train_batch_size = 2

fealoaders = get_dataloader(data_type='fea', head_type = head_type, h5_paths = h5_paths, batch_size = train_batch_size)


In [68]:
continue_training = True
lr = 1e-4

head = Head(head_type,fealoaders[0].dataset.channel_size,dropout)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(head.parameters(),lr = lr)
latest_ep = 0

if continue_training:
    head,optimizer,latest_ep = get_model_from_ckpt(folder, head, model_type = 'head', metric = 'ep',optimizer = optimizer)


In [18]:
epochs = 20
patience = 5
mini_iter = 2

head.to(device)

prev_srcc = 0
prev_plcc = 0
patience_cnt = 0

for epoch in range(epochs): 
    epoch = int(latest_ep)+epoch
    head = train(head,fealoaders[0],device,loss_fn,optimizer,mini_iter,epoch)
    plcc, srcc, acc, val_loss = eval(head,fealoaders[1],loss_fn,device)
    logging(epoch, val_loss,acc,plcc,srcc)

    if (plcc<prev_plcc)&(srcc<prev_srcc):
        patience_cnt+=1
    else:
        patience_cnt = 0
        print('Saving ckpt')
        save_model(head,optimizer,folder,epoch,plcc,srcc,acc,'head')
        prev_plcc = plcc
        prev_srcc = srcc

    optimizer = lr_scheduler(optimizer, epoch, lr, patience_cnt)

    if patience_cnt == patience:
        if lr == 1e-6:
            print('CONVERGE')
            break
        else:
            patience_cnt = 0
    



Epoch 0 batch0 train_loss: 0.10825064033269882
Epoch 0: val loss-0.1902375552803278 val acc-[0.66850994], srcc-[0.02912735], plcc-[0.10578222]
Saving ckpt
Epoch 1 batch0 train_loss: 0.206095352768898
Epoch 1: val loss-0.19199119992554187 val acc-[0.44167432], srcc-[0.140795], plcc-[0.78823808]
Saving ckpt
Epoch 2 batch0 train_loss: 0.13108108937740326


KeyboardInterrupt: 

In [ ]:
head,_,_ = get_model_from_ckpt(folder,head,'head','srcc')

In [13]:

plcc, srcc, acc, val_loss = eval(head,fealoaders[2],loss_fn,device)
print(f'Test loss-{val_loss}')
print(f'Test acc-{acc}, srcc-{srcc}, plcc-{plcc}')  


Test loss-0.15837158039212226
Test acc-[0.00844305], srcc-[0.47571653], plcc-[0.71483876]
